# Практическое задание 2

**Построение графа друзей из VK**

1. Создадим класс пользователя ВК (вершины графа). Пользователь задается двумя параметрами:

- VK ID
- Список друзей (их VK ID)

In [77]:
class vk_user:
    def __init__(self, vk_id, friends):
        self.vk_id = vk_id
        self.friends = friends

Создадим список "корневых" пользователей (то есть друзей глубины 0):

In [78]:
users = [vk_user(229180632, None)]

По данному списку пройдемся дважды: в первый раз будут определены друзья для корневых пользователей, во второй раз - друзья их друзей.

In [79]:
import requests

deep = 2
for _ in range(deep):
    new_users = []
    for user in users:
        if user.friends == None:
            url = f'https://api.vk.com/method/friends.get?user_id={user.vk_id}&&v=5.199'
            response = requests.get(url)
            if 'response' not in response.json():
                continue
            user.friends = response.json()['response']['items']
            for friend in response.json()['response']['items']:
                if all(person.vk_id != friend for person in users):
                    new_users.append(vk_user(friend, None))
    users.extend(new_users)


Поскольку накладно каждый раз обращаться к VK API, данные будут выгружены один раз и скомпанованы в CSV-файл.

In [80]:
import pandas as pd

friendship_tuples = []

for user in users:
    if user.friends == None:
        continue
    for friend in user.friends:
        if (friend, user.vk_id) not in friendship_tuples:
            friendship_tuples.append((user.vk_id, friend))


df = pd.DataFrame(friendship_tuples, columns=['user1', 'user2'])
df.to_csv('friendships.csv', index=False)